In [51]:
import pandas as pd
import requests
import ast
from os import environ 

import numpy as np
import datetime
import json

from os import listdir
from os.path import isfile, join



In [68]:

#base_dir = "data"

#base_dir = join("Vaults", "aEval_70")

#dirr = join(base_dir, "05_model_input/")


base_dir = join("Vaults", "aaaaaSN2books")

#base_dir = "data"

dirr = join(base_dir, "05_model_input/")



In [69]:
raw_text_df = pd.read_csv(dirr+"text_df_final.csv", index_col=0)

text_df = pd.read_csv(dirr+"text_df_final.csv", index_col=0)

#text_df = pd.read_csv("data/01_raw/text_df.csv", index_col=0)

mentions_df = pd.read_csv(dirr+"mentions_df_final.csv")

mentions_df.sort_values("Start_pos", ascending=False, inplace=True)


relatedness_df = pd.read_csv(dirr+"relatedness_df.csv")


#concepts_df = pd.read_csv(dirr+"concepts_database_enhanced.csv", index_col=0)


with open(dirr+"conceptnet_to_dbpedia.json", "r") as f:
        conceptnet_to_dbpedia =  json.load(f)

with open(dirr+"dbpedia_to_conceptnet.json", "r") as f:
        dbpedia_to_conceptnet =  json.load(f)


with open(dirr+"concepts_list_final.json", "r") as f:
        concepts_list = json.load(f)



In [70]:
conceptsdir = join(base_dir, "04_feature")

with open(join(join(base_dir, "03_primary"),"concepts_data_list.json"), "r") as f:
        concepts_data_list = json.load(f)

with open(join(conceptsdir,"conceptnet_distances_dict.json"), "r") as f:
        conceptnet_distances_dict = json.load(f)

with open(join(conceptsdir,"conceptnet_relations_dict.json"), "r") as f:
        conceptnet_relations_dict = json.load(f)



new_concepts_data_list = []
for data in concepts_data_list:
	if type(data) != type([0]):
		print("Error")
		print(data)
	else:
		new_concepts_data_list.append(data)


column_names = ["DBpedia URL", "Name", "Abstract", "Image", "DBpedia Types", "Subject FROM", "Related Entities FROM", "Related Entities OF", "Subject OF", "Narrower Categories", "Broader Categories"] 

concept_df = pd.DataFrame(new_concepts_data_list, columns=column_names)

concepts_database = concept_df[concept_df["DBpedia URL"].isin(concepts_list)]


relations_dict_column = []
distances_dict_column = []

for row in concepts_database.itertuples():

	concept_id = row[1]

	distances_dict = conceptnet_distances_dict[concept_id]

	relations_dict = conceptnet_relations_dict[concept_id]

	distances_dict_column.append(distances_dict)

	relations_dict_column.append(relations_dict)

concepts_database_enhanced = concepts_database.copy()

concepts_database_enhanced['ConceptNet_Distances'] = distances_dict_column

concepts_database_enhanced['ConceptNet_Relations'] = relations_dict_column

concepts_df = concepts_database_enhanced

concepts_list = list(concepts_df["DBpedia URL"])
print(len(concepts_list))

concepts_df = concepts_df.set_index("DBpedia URL")

71


In [71]:
print(len(concepts_list))
ignore_concept = []
for row in mentions_df.itertuples():
	types = row[14]
	concept_url = row[10]
	if "http://dbpedia.org/ontology/Person" in types:
		ignore_concept.append(concept_url)

new_concept_list = []
for concept in concepts_list:
	concept_net_dict = concepts_df.loc[concept,"ConceptNet_Relations"] #ast.literal_eval(concepts_df.loc[concept,"ConceptNet_Relations"])

	if concept not in ignore_concept and concept_net_dict['/r/RelatedTo'] != []:
		new_concept_list.append(concept)

concepts_list = new_concept_list
print(len(concepts_list))

71
61



## Read DBpedia Relation Matrix

In [72]:
concept_links = {}

for concept_url in concepts_list:
	concept_links[concept_url] = {'link_from': [], 'link_to': [], 'any_link': [], 'both_links': [], 'shared_subject':{}}
	#print(concept_url)
	#print(concepts_df_dbpedia_index.loc[concept_url, "Name"])

for concept_url in concepts_list:

	concepts_linked_from_this_concept = concepts_df.loc[concept_url, "Related Entities FROM"]
	this_concept_is_linked_from = concepts_df.loc[concept_url, "Related Entities OF"]
	subject_list = concepts_df.loc[concept_url, "Subject FROM"]

	for other_concept in concepts_list:
		if other_concept in concepts_linked_from_this_concept:
			concept_links[other_concept]['link_from'].append(concept_url)

		if other_concept in this_concept_is_linked_from:
			concept_links[other_concept]['link_to'].append(concept_url)

		if other_concept in concepts_linked_from_this_concept or other_concept in this_concept_is_linked_from:
			concept_links[other_concept]['any_link'].append(concept_url)

		if other_concept in concepts_linked_from_this_concept and other_concept in this_concept_is_linked_from:
			concept_links[other_concept]['both_links'].append(concept_url)

		other_subject_list = concepts_df.loc[other_concept, "Subject FROM"]
		if concept_url != other_concept:
			for subject in other_subject_list:

		
				if subject in subject_list:

					if concept_url in list(concept_links[other_concept]['shared_subject'].keys()):

						concept_links[other_concept]['shared_subject'][concept_url].append(subject)
					else: 
						concept_links[other_concept]['shared_subject'][concept_url]= [subject]
	

In [73]:

concepts_distance_matrix_links = []
concepts_distance_matrix_subject = []

for concept in concepts_list:
	
	new_row_link = []
	new_row_subject = []

	any_link_list = concept_links[concept]['any_link']
	shared_subject_list = concept_links[concept]['shared_subject']

	for concept2 in concepts_list:
		if concept == concept2:
			score_link = 1
		elif concept2 in any_link_list:
			score_link = 1
		else: #concept not related
			score_link = 0
		new_row_link.append(score_link)

		if concept == concept2:
			score_subject = 1
		elif concept2 in shared_subject_list:
			score_subject = 1
		else: 
			score_subject = 0
		new_row_subject.append(score_subject)

	
	
	concepts_distance_matrix_links.append(new_row_link)

	concepts_distance_matrix_subject.append(new_row_subject)


concepts_distance_hyperlinks = pd.DataFrame(concepts_distance_matrix_links, columns = concepts_list)

concepts_distance_subject = pd.DataFrame(concepts_distance_matrix_subject, columns = concepts_list)




dbpedia_link_matrix = concepts_distance_hyperlinks
dbpedia_link_matrix.index = concepts_list




In [74]:
#dbpedia_link_matrix.head()

## Read ConceptNet Distance Matrix

In [75]:

def build_conceptnet_distance_matrix(concepts_list, concepts_df):

        concepts_distance_matrix = []

        for concept_url in concepts_list:

                new_row_link = []
                distance_dict = concepts_df.loc[concept_url]["ConceptNet_Distances"]#ast.literal_eval(concepts_df.loc[concept_url]["ConceptNet_Distances"])

                for concept_url2 in concepts_list:
                        
                        distance = distance_dict[concept_url2]
                        new_row_link.append(distance)

                concepts_distance_matrix.append(new_row_link)
        
        distance_matrix = pd.DataFrame(concepts_distance_matrix, columns = concepts_list)        

        return distance_matrix


In [76]:

relatedness_matrix_cn = build_conceptnet_distance_matrix(concepts_list, concepts_df)
relatedness_matrix_cn.index = concepts_list
np.fill_diagonal(relatedness_matrix_cn.values, 0)

def min_max_scaling(series, max_value, min_value):
	
    return (series - min_value) / (max_value - min_value)

def normalize_df(df):
	max_value = 0
	min_value = 0
	for col in df.columns:
		max = df[col].max()
		min = df[col].min()
		if max > max_value:
			max_value = max
		if min < min_value:
			min_value = min
	print(max_value, min_value)
	for col in df.columns:
    		df[col] = min_max_scaling(df[col], max_value, min_value)

	return df


relatedness_matrix_cn = normalize_df(relatedness_matrix_cn)
np.fill_diagonal(relatedness_matrix_cn.values, 1)

1.0 -0.157


In [77]:
relatedness_matrix_cn.head(10)

,http://dbpedia.org/resource/Thought,http://dbpedia.org/resource/Mind,http://dbpedia.org/resource/Internet,http://dbpedia.org/resource/Information,http://dbpedia.org/resource/System,http://dbpedia.org/resource/Experience,http://dbpedia.org/resource/Scientific_method,http://dbpedia.org/resource/Knowledge,http://dbpedia.org/resource/Wealth,http://dbpedia.org/resource/Person,...,http://dbpedia.org/resource/Chaos_theory,http://dbpedia.org/resource/Randomness,http://dbpedia.org/resource/Homeostasis,http://dbpedia.org/resource/Goal,http://dbpedia.org/resource/Nervous_system,http://dbpedia.org/resource/Abstraction,http://dbpedia.org/resource/Correlation,http://dbpedia.org/resource/Insight,http://dbpedia.org/resource/Machine,http://dbpedia.org/resource/Social_network
http://dbpedia.org/resource/Thought,1.000000,0.433881,0.148660,0.137424,0.205704,0.223855,0.151253,0.239412,0.169404,0.160761,...,0.108902,0.064823,0.177182,0.238548,0.249784,0.233362,0.110631,0.310285,0.148660,0.163354
http://dbpedia.org/resource/Mind,0.433881,1.000000,0.171997,0.155575,0.167675,0.203976,0.187554,0.260156,0.177182,0.155575,...,0.178047,0.094209,0.289542,0.265341,0.274849,0.170268,0.177182,0.282627,0.189283,0.093345
http://dbpedia.org/resource/Internet,0.148660,0.171997,1.000000,0.313742,0.171132,0.167675,0.128781,0.231634,0.219533,0.206569,...,0.150389,0.244598,0.129646,0.140017,0.127917,0.155575,0.169404,0.101988,0.160761,0.473639
http://dbpedia.org/resource/Information,0.137424,0.155575,0.313742,1.000000,0.171997,0.259291,0.203976,0.533276,0.378565,0.254970,...,0.159896,0.193604,0.170268,0.109767,0.133967,0.146067,0.205704,0.370787,0.165082,0.239412
http://dbpedia.org/resource/System,0.205704,0.167675,0.171132,0.171997,1.000000,0.206569,0.236819,0.184961,0.171132,0.179775,...,0.116681,0.170268,0.143475,0.160761,0.306828,0.050130,0.108038,0.101988,0.387208,0.208297
http://dbpedia.org/resource/Experience,0.223855,0.203976,0.167675,0.259291,0.206569,1.000000,0.131374,0.532411,0.286949,0.280035,...,0.174589,0.119274,0.168539,0.146067,0.124460,0.121867,0.195333,0.384615,0.203976,0.150389
http://dbpedia.org/resource/Scientific_method,0.151253,0.187554,0.128781,0.203976,0.236819,0.131374,1.000000,0.216076,0.108902,0.108038,...,0.332757,0.115817,0.272256,0.155575,0.214347,0.166811,0.245462,0.168539,0.257563,0.146067
http://dbpedia.org/resource/Knowledge,0.239412,0.260156,0.231634,0.533276,0.184961,0.532411,0.216076,1.000000,0.475367,0.173725,...,0.150389,0.170268,0.204840,0.145203,0.159032,0.151253,0.178911,0.503025,0.163354,0.165946
http://dbpedia.org/resource/Wealth,0.169404,0.177182,0.219533,0.378565,0.171132,0.286949,0.108902,0.475367,1.000000,0.068280,...,0.173725,0.242869,0.139153,0.146932,0.142610,0.091616,0.237684,0.265341,0.099395,0.147796
http://dbpedia.org/resource/Person,0.160761,0.155575,0.206569,0.254970,0.179775,0.280035,0.108038,0.173725,0.068280,1.000000,...,0.206569,0.125324,0.395851,0.077787,0.189283,0.102852,0.123596,0.127917,0.257563,0.220398


## Read ConceptNet Relation Matrix

In [78]:
relation_list = ["/r/RelatedTo",
		 "/r/IsA",
		 "/r/HasContext",
		  "/r/Causes",
		  "/r/CapableOf",
		   "/r/MotivatedByGoal",
		   "/r/Desires",
		    "/r/HasProperty",
		     "/r/HasPrerequisite",
		      "/r/PartOf",
		       "/r/UsedFor",
		        "/r/Antonym",
			"/r/DistinctFrom",
			 "/r/Synonym",
			  "/r/SimilarTo"]

relation_list = ["/r/RelatedTo", "/r/IsA", "/r/HasContext", "/r/Causes", "/r/CapableOf", "/r/MotivatedByGoal", "/r/HasProperty", "/r/HasPrerequisite", "/r/PartOf",  "/r/Antonym", "/r/DistinctFrom", "/r/Synonym", "/r/SimilarTo"]
				

In [79]:

def build_conceptnet_relations_matrix(relation_list, concepts_list, concepts_df):
	"""
	for relation in relation_list:
		
		concepts_distance_matrix = []
		
		for concept_url in concepts_list:

			conceptNet_relations_dict = ast.literal_eval(concepts_df.loc[concept_url]["ConceptNet_Relations"])

			related_concepts = [relation]
			new_row_link = []

			for concept_url2 in concepts_list:

				concept2 = dbpedia_to_conceptnet[concept_url2]
				
				if concept2 in related_concepts:
					has_link = 1
				else: 
					has_link = 0

				new_row_link.append(has_link)
			concepts_distance_matrix.append(new_row_link)
		#distance_matrix = pd.DataFrame(concepts_distance_matrix, columns = concepts_list)
	"""
	concepts_distance_matrix = []

	for concept_url in concepts_list:
	
		new_row_link = []

		any_link_list_1 = concept_links[concept_url]['any_link']

		conceptNet_relations_dict_1 = concepts_df.loc[concept_url]["ConceptNet_Relations"] #ast.literal_eval(concepts_df.loc[concept_url]["ConceptNet_Relations"])
		concept = dbpedia_to_conceptnet[concept_url]
		#print(concept)

		for concept_url2 in concepts_list:


			if concept_url == concept_url2:
				has_link=1
			else:
				has_link=0

			#any_link_list_2 = concept_links[concept_url_2]['any_link']

			conceptNet_relations_dict_2 = concepts_df.loc[concept_url2]["ConceptNet_Relations"]#ast.literal_eval(concepts_df.loc[concept_url2]["ConceptNet_Relations"])
			concept2 = dbpedia_to_conceptnet[concept_url2]

			if concept_url2 in any_link_list_1:
				has_link = 1

			for relation in relation_list:

				related_concepts_1 = conceptNet_relations_dict_1[relation]
				
				related_concepts_2 = conceptNet_relations_dict_2[relation]
			
				if concept2 in related_concepts_1:
					has_link = 1

				if concept in related_concepts_2:
					has_link = 1

			new_row_link.append(has_link)
			
		concepts_distance_matrix.append(new_row_link)

	distance_matrix = pd.DataFrame(concepts_distance_matrix, columns = concepts_list)

	return distance_matrix


In [80]:

any_relation_matrix = build_conceptnet_relations_matrix(relation_list, concepts_list, concepts_df)
any_relation_matrix.index = concepts_list

In [81]:
any_relation_matrix.head(10)

,http://dbpedia.org/resource/Thought,http://dbpedia.org/resource/Mind,http://dbpedia.org/resource/Internet,http://dbpedia.org/resource/Information,http://dbpedia.org/resource/System,http://dbpedia.org/resource/Experience,http://dbpedia.org/resource/Scientific_method,http://dbpedia.org/resource/Knowledge,http://dbpedia.org/resource/Wealth,http://dbpedia.org/resource/Person,...,http://dbpedia.org/resource/Chaos_theory,http://dbpedia.org/resource/Randomness,http://dbpedia.org/resource/Homeostasis,http://dbpedia.org/resource/Goal,http://dbpedia.org/resource/Nervous_system,http://dbpedia.org/resource/Abstraction,http://dbpedia.org/resource/Correlation,http://dbpedia.org/resource/Insight,http://dbpedia.org/resource/Machine,http://dbpedia.org/resource/Social_network
http://dbpedia.org/resource/Thought,1,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,1,0,0
http://dbpedia.org/resource/Mind,1,1,0,0,0,1,0,1,0,1,...,0,0,0,0,1,0,0,0,1,0
http://dbpedia.org/resource/Internet,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
http://dbpedia.org/resource/Information,0,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
http://dbpedia.org/resource/System,0,0,0,1,1,0,0,0,0,1,...,1,0,1,1,1,0,0,0,0,0
http://dbpedia.org/resource/Experience,1,1,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Scientific_method,0,0,0,0,0,0,1,1,1,0,...,0,0,0,0,0,0,1,0,1,0
http://dbpedia.org/resource/Knowledge,1,1,0,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Wealth,0,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
http://dbpedia.org/resource/Person,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# Find Distance Between Texts

In [82]:
texts_concepts = {}
texts_concepts_repeated = {}
drop_list = []
for text_id in text_df.index:
	
	concepts_in_text = []
	concepts_in_text_w_repeat = []
	

	for row in mentions_df.itertuples():
		id = row[1]		
		if id == text_id:
			concept_url = row[10]
			if concept_url in concepts_list:
				concepts_in_text_w_repeat.append(concept_url)
				if concept_url not in concepts_in_text and concept_url != "http://dbpedia.org/resource/Ancient_Greek":
					concepts_in_text.append(concept_url)

	if len(concepts_in_text) == 0:
		drop_list.append(text_id)
	texts_concepts[text_id]=concepts_in_text
	texts_concepts_repeated[text_id]=concepts_in_text_w_repeat

print(drop_list)

[547332531, 547332532, 547332538, 583516122, 591548156]


## Calculate Transition Matrixes

In [83]:
"""transition_matrixes_list = []

for distance_matrix in matrix_list:

	transition_matrix_array = []

	for concept_url in concepts_list:

		new_row = []

		concept_weight_sum = distance_matrix[concept_url].sum() 

		for concept_url2 in concepts_list:

			distance = distance_matrix.loc[concept_url,concept_url2]
			
			transition_score = distance/concept_weight_sum

			new_row.append(transition_score)


		transition_matrix_array.append(new_row)
	
	transition_matrixes_list.append(pd.DataFrame(transition_matrix_array, columns = concepts_list))
"""

'transition_matrixes_list = []\n\nfor distance_matrix in matrix_list:\n\n\ttransition_matrix_array = []\n\n\tfor concept_url in concepts_list:\n\n\t\tnew_row = []\n\n\t\tconcept_weight_sum = distance_matrix[concept_url].sum() \n\n\t\tfor concept_url2 in concepts_list:\n\n\t\t\tdistance = distance_matrix.loc[concept_url,concept_url2]\n\t\t\t\n\t\t\ttransition_score = distance/concept_weight_sum\n\n\t\t\tnew_row.append(transition_score)\n\n\n\t\ttransition_matrix_array.append(new_row)\n\t\n\ttransition_matrixes_list.append(pd.DataFrame(transition_matrix_array, columns = concepts_list))\n'

## Concepts Distance Matrix A_l

In [84]:
texts_list = []
for item in list(text_df.index):
	if item not in drop_list:
		texts_list.append(item)


In [85]:

max_distance = []
max_relation = []

avg_distance = []
avg_relation = []

avg_distance_repeat = []
avg_relation_repeat = []



for text in texts_list:


	concepts = texts_concepts[text]
	concepts_repeat = texts_concepts_repeated[text]

	max_row_dist = []
	max_row_rel = []


	avg_row_dist = []
	avg_row_rel = []

	avg_row_dist_repeat = []
	avg_row_rel_repeat = []

	for text2 in texts_list:

		concepts2 = texts_concepts[text2]
		concepts_repeat2 = texts_concepts_repeated[text2]

		
		max_sim_dist = 0
		max_sim_rel = 0


		total_sim_dist = 0 
		dist_count = 0.00000000000000001
		dist_count = 1.5

		total_sim_dist_repeat = 0.00000000000000001
		dist_count_repeat = 1.5


		total_sim_rel = 0
		rel_count = 0.00000000000000001
		rel_count = 1.5

		total_sim_rel_repeat = 0.00000000000000001
		rel_count_repeat = 1.5

		for concept1 in concepts:
			
			for concept2 in concepts2:
				
				sim_dist = max(relatedness_matrix_cn.loc[concept1,concept2], relatedness_matrix_cn.loc[concept2,concept1])
				
				max_sim_dist = max(max_sim_dist, sim_dist)
				total_sim_dist += sim_dist
				dist_count +=0.95


				sim_rel = max(any_relation_matrix.loc[concept1,concept2], any_relation_matrix.loc[concept2,concept1])

				max_sim_rel = max(max_sim_rel, sim_rel)
				total_sim_rel += sim_rel

				rel_count +=0.95


		for concept1 in concepts_repeat:
			
			for concept2 in concepts_repeat2:
				
				sim_dist = max(relatedness_matrix_cn.loc[concept1,concept2], relatedness_matrix_cn.loc[concept2,concept1])
				
				total_sim_dist_repeat += sim_dist
				
				dist_count_repeat +=0.95


				sim_rel = max(any_relation_matrix.loc[concept1,concept2], any_relation_matrix.loc[concept2,concept1])

				total_sim_rel_repeat += sim_rel

				rel_count_repeat +=0.95


		max_row_dist.append(max_sim_dist)
		max_row_rel.append(max_sim_rel)


		avg_row_dist.append(round(total_sim_dist/dist_count, 4))
		avg_row_dist_repeat.append(round(total_sim_dist_repeat/dist_count_repeat, 4))
		
		avg_row_rel.append(round(total_sim_rel/rel_count, 4))
		avg_row_rel_repeat.append(round(total_sim_rel_repeat/rel_count_repeat, 4))
	
	max_distance.append(max_row_dist)
	max_relation.append(max_row_rel)

	avg_distance.append(avg_row_dist)
	avg_relation.append(avg_row_rel)

	avg_distance_repeat.append(avg_row_dist_repeat)
	avg_relation_repeat.append(avg_row_rel_repeat)

	

In [86]:
matrix_max_dist = pd.DataFrame(max_distance, index=texts_list, columns=texts_list)

matrix_max_rel = pd.DataFrame(max_relation, index=texts_list, columns=texts_list)



matrix_avg_dist = pd.DataFrame(avg_distance, index=texts_list, columns=texts_list)

matrix_avg_rel = pd.DataFrame(avg_relation, index=texts_list, columns=texts_list)



matrix_avg_dist_repeat = pd.DataFrame(avg_distance_repeat, index=texts_list, columns=texts_list)

matrix_avg_rel_repeat = pd.DataFrame(avg_relation_repeat, index=texts_list, columns=texts_list)


matrix_avg_dist = normalize_df(matrix_avg_dist)
np.fill_diagonal(matrix_avg_dist.values, 1)

matrix_avg_rel = normalize_df(matrix_avg_rel)
np.fill_diagonal(matrix_avg_rel.values, 1)

matrix_avg_dist_repeat = normalize_df(matrix_avg_dist_repeat)
np.fill_diagonal(matrix_avg_dist_repeat.values, 1)

matrix_avg_rel_repeat = normalize_df(matrix_avg_rel_repeat)
np.fill_diagonal(matrix_avg_rel_repeat.values, 1)


final_matrix = matrix_avg_dist.multiply(0.6).add(matrix_avg_rel.multiply(0.4))

final_matrix_2 = matrix_avg_dist.multiply(0.25).add(matrix_avg_rel.multiply(0.25)).add(matrix_max_dist.multiply(0.25)).add(matrix_max_rel.multiply(0.25))

final_matrix_3 = matrix_avg_dist_repeat.multiply(0.6).add(matrix_avg_rel_repeat.multiply(0.4))



0.5716 0
0.8955 0
0.8162 0
0.8955 0


In [87]:
#matrix_max_dist.head(10)

In [88]:
#matrix_avg_rel.head(10)

In [89]:
final_matrix.head()

,514356011,514356012,514356013,514356014,514356015,514356016,514356017,514356018,514356019,514356020,...,591548155,591548157,591548158,591548159,591548160,591548161,591548162,591548163,591548164,591548165
514356011,1.000000,0.091533,0.260950,0.277781,0.209287,0.195871,0.201757,0.187454,0.257553,0.184955,...,0.341147,0.218429,0.176767,0.202053,0.270951,0.243684,0.237189,0.272093,0.258851,0.345093
514356012,0.091533,1.000000,0.232504,0.197515,0.207818,0.382144,0.176662,0.231331,0.129846,0.438814,...,0.250944,0.158177,0.132785,0.170784,0.239936,0.099300,0.212856,0.130371,0.074633,0.121344
514356013,0.260950,0.232504,1.000000,0.459306,0.480152,0.345825,0.345991,0.329142,0.374973,0.316650,...,0.334254,0.310712,0.300885,0.331286,0.514749,0.081666,0.385910,0.489497,0.475714,0.553389
514356014,0.277781,0.197515,0.459306,1.000000,0.365032,0.277691,0.378683,0.243176,0.415417,0.302158,...,0.323728,0.302690,0.247679,0.342138,0.440748,0.202273,0.372695,0.431692,0.357312,0.446207
514356015,0.209287,0.207818,0.480152,0.365032,1.000000,0.330010,0.345332,0.263422,0.343464,0.281845,...,0.318392,0.241363,0.251615,0.331739,0.380838,0.231440,0.343884,0.449273,0.363237,0.424434


In [90]:
final_matrix_3.head()

,514356011,514356012,514356013,514356014,514356015,514356016,514356017,514356018,514356019,514356020,...,591548155,591548157,591548158,591548159,591548160,591548161,591548162,591548163,591548164,591548165
514356011,1.000000,0.064102,0.193629,0.252145,0.146208,0.139010,0.159887,0.143445,0.192220,0.129527,...,0.283045,0.150428,0.127689,0.195517,0.200096,0.269507,0.172378,0.235458,0.206538,0.326968
514356012,0.064102,1.000000,0.252585,0.145724,0.158558,0.310385,0.123720,0.174173,0.098579,0.343813,...,0.190452,0.113842,0.093286,0.121808,0.219867,0.089169,0.156562,0.103210,0.052267,0.090787
514356013,0.193629,0.252585,1.000000,0.368371,0.400962,0.366644,0.320126,0.296070,0.354289,0.308629,...,0.219492,0.251885,0.220021,0.286782,0.539731,0.078510,0.458695,0.472164,0.315952,0.458600
514356014,0.252145,0.145724,0.368371,1.000000,0.253732,0.209399,0.269962,0.163388,0.352259,0.231082,...,0.262696,0.223680,0.183556,0.267676,0.374755,0.172581,0.315640,0.329651,0.315316,0.345423
514356015,0.146208,0.158558,0.400962,0.253732,1.000000,0.269934,0.277896,0.203036,0.266751,0.218102,...,0.255930,0.198487,0.201600,0.285030,0.337214,0.281862,0.307978,0.335446,0.259962,0.302322


In [91]:
#text_df.loc[final_matrix.index][["Text_ID", "Author_name"]]

pd.concat([text_df.loc[final_matrix.index][["Text_ID", "Author_name"]], final_matrix], axis=1).to_csv(dirr + "relatedness_concepts_df.csv")

pd.concat([text_df.loc[final_matrix.index][["Text_ID", "Author_name"]], final_matrix_3], axis=1).to_csv(dirr + "relatedness_concepts_df_repeat.csv")

# Calculating Coherence

In [92]:
drop_list_str =[]

for item in drop_list:
	drop_list_str.append(str(item))

In [93]:
relatedness_df.set_index("Text_ID", drop=True, inplace=True)
semantic_similarities_matrix = relatedness_df.drop(["Father_ID", "Father_name", "Author_name", "Text"], axis =1).drop(drop_list_str, axis=1).drop(drop_list, axis=0)

semantic_similarities_matrix.head()

,514356011,514356012,514356013,514356014,514356015,514356016,514356017,514356018,514356019,514356020,...,591548155,591548157,591548158,591548159,591548160,591548161,591548162,591548163,591548164,591548165
Text_ID,,,,,,,,,,,,,,,,,,,,,
514356011,1.00000,0.30399,0.51498,0.31325,0.35491,0.29539,0.41681,0.37155,0.48790,0.34338,...,0.12798,0.32314,0.34290,0.51134,0.57004,0.58593,0.42383,0.52415,0.76503,0.50365
514356012,0.30399,1.00000,0.45006,0.20825,0.42576,0.69211,0.18882,0.64008,0.20262,0.69130,...,0.00784,0.12592,0.13305,0.17750,0.37170,0.22233,0.36963,0.17383,0.29004,0.26405
514356013,0.51498,0.45006,1.00000,0.27488,0.60716,0.37646,0.36172,0.45966,0.36758,0.40818,...,0.15242,0.33138,0.32864,0.40841,0.47662,0.40982,0.44512,0.35258,0.58804,0.45933
514356014,0.31325,0.20825,0.27488,1.00000,0.17432,0.23667,0.32379,0.28304,0.46971,0.25023,...,0.10094,0.13806,0.12520,0.21060,0.47082,0.34604,0.39306,0.30591,0.24234,0.30340
514356015,0.35491,0.42576,0.60716,0.17432,1.00000,0.36721,0.29451,0.37987,0.24669,0.32848,...,0.10145,0.30211,0.25304,0.37370,0.38479,0.36500,0.38651,0.18824,0.38189,0.32524


## Read Semantic Relatedness Matrix

In [94]:


def min_max_scaling(series, max_value, min_value):
	
    return (series - min_value) / (max_value - min_value)

def normalize_df(df):
	max_value = 0
	min_value = 0
	for col in df.columns:
		max = df[col].max()
		min = df[col].min()
		if max > max_value:
			max_value = max
		if min < min_value:
			min_value = min

	for col in df.columns:
    		df[col] = min_max_scaling(df[col], max_value, min_value)

	return df

norm_semantic = normalize_df(semantic_similarities_matrix)



In [95]:
#norm_semantic.head()

In [96]:
import mantel

In [97]:
upper_semantic = []
i=0
for row in norm_semantic.itertuples():
	i+=1
	for j in range(i,len(norm_semantic)+1):
		upper_semantic.append(row[j])


upper_concepts = []
i=0
for row in final_matrix.itertuples():
	i+=1
	for j in range(i,len(final_matrix)+1):
		upper_concepts.append(row[j])


upper_concepts_2 = []
i=0
for row in final_matrix_3.itertuples():
	i+=1
	for j in range(i,len(final_matrix)+1):
		upper_concepts_2.append(row[j])

print(mantel.test(upper_concepts, upper_semantic, perms=10000, method='pearson'))

print(mantel.test(upper_concepts_2, upper_semantic, perms=10000, method='pearson'))



MantelResult(0.6360398393807076, 0.0001, 21.177436447153333)
MantelResult(0.663533190050432, 0.0001, 22.53659293903043)


In [98]:
"""Courage_Fear

0.55

0.673
0.679

0.60

0.653
0.665


0.65

0.642
0.642

0.70

0.675
0.681


Book Selection

0.55

0.500
0.519


0.60

0.479
0.499


0.65

0.489
0.508


0.70

0.469
0.481

"""

'Courage_Fear\n\n0.55\n\n0.673\n0.679\n\n0.60\n\n0.653\n0.665\n\n\n0.65\n\n0.642\n0.642\n\n0.70\n\n0.675\n0.681\n\n\nBook Selection\n\n0.55\n\n0.500\n0.519\n\n\n0.60\n\n0.479\n0.499\n\n\n0.65\n\n0.489\n0.508\n\n\n0.70\n\n0.469\n0.481\n\n'